# Setup

In [1]:
import os
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

from select_disease_cohort import *
from select_patient_info import *

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_curve, auc, RocCurveDisplay


# Constants
GROUP_COL = 'subject_id' # Identifier for patients
VISIT_COL = 'hadm_id' # Identifier for visits
ADMIT_COL = 'admittime' # Admission time
DISCH_COL = 'dischtime' # Discharge time
GAP = datetime.timedelta(days=14) # Gap for within readmission must occur
DISEASE_LABEL = 'I50' # or I50 = Heart Failure, # I25 = Coronary Artery Disease, # N18 = Chronic Kidney Disease, # J44 = Chronic obstructive pulmonary disease
VERSION = '1.0' # MIMIC-IV version 1.0 or 2.0

# Results so far for READMISSION PREDICTION
# I25, GAP=14, MIMIC-IV 2.0, --> ROC-AUC = .57
# I25, GAP=7, MIMIC-IV 2.0, --> ROC-AUC = .52
# I550, GAP=7, MIMIC-IV 2.0, --> ROC-AUC = .46
# I550, GAP=30, MIMIC-IV 2.0, --> ROC-AUC = .56

# Results so far for DEATH PREDICTION
# I50, GAP=14, MIMIC-IV 2.0, --> ROC-AUC = .80
# I25, GAP=14, MIMIC-IV 2.0, --> ROC-AUC = .66

# Paths
PATH_ADMISSIONS: str = os.path.join("mimiciv", VERSION, "core", "admissions.csv.gz")
PATH_DIAGNOSES_ICD: str = os.path.join("mimiciv", VERSION, "hosp", "diagnoses_icd.csv.gz")
PATH_PATIENTS: str = os.path.join("mimiciv", VERSION, "core", "patients.csv.gz")
PATH_OMR: str = os.path.join("mimiciv", VERSION, "hosp", "omr.csv.gz")
PATH_PRESCRIPTIONS: str = os.path.join("mimiciv", VERSION, "hosp", "prescriptions.csv.gz")
PATH_ICD_MAP: str = os.path.join("utils", "ICD9_to_ICD10_mapping.txt")

# Loading admissions, selecting disease cohort

In [ ]:
# From admissions df, standardize ICD codes (9 and 10 are mixed) and select disease cohort based on disease label (ICD10)
visit_df = standardize_codes_and_select_cohort(disease_label=DISEASE_LABEL, path_admissions_df=PATH_ADMISSIONS, filter_deaths=False)

# Adding patient df info (and basic vars from omr in case of MIMIC-IV 2.0)

In [3]:
# Appends data from patients_df to visit_df, e.g. age, gender, ethnicity, marital status
# And also comorbidities
visit_pts_df = get_patient_df_info(version=VERSION, visit_df=visit_df)
visit_pts_df = get_comorbidities(get_diagnosis_icd_df(), visit_pts_df)
visit_pts_df = visit_pts_df.sort_values(['subject_id', 'admittime']).reset_index(drop=True)

In [4]:
# Adding omr basic physical variables via row-wise operations
visit_pts_df['bmi'] = visit_pts_df.apply(find_closest_bmi, axis=1)
visit_pts_df.dropna(subset=['bmi'],inplace=True) # Has a lot of missing values, because omr = outpatient measurements 

# Create cols for systolic and diastolic blood pressure
visit_pts_df['bp_systolic'] = visit_pts_df.apply(find_closest_bp_systolic, axis=1)
visit_pts_df['bp_diastolic'] = visit_pts_df.apply(find_closest_bp_diastolic, axis=1)

visit_pts_df.dropna(subset=['bp_systolic'],inplace=True) # Has a lot of missing values, because omr = outpatient measurements 
visit_pts_df.dropna(subset=['bp_diastolic'],inplace=True) 

# Adding number of medications per visit. Has no missings as it is lookup over hadm_id
visit_pts_df['presc_meds'] = visit_pts_df.apply(get_med_prescs, axis=1)

# ML Tasks

## Death prediction

In [5]:
# Task spec
task = "death_pred"
# Creating targets and balancing set)
visit_pts_df.loc[visit_pts_df['hospital_expire_flag'] == 1, 'y_label'] = 1
visit_pts_df.loc[visit_pts_df['dod'].isna(), 'y_label'] = 0
visit_pts_df.loc[visit_pts_df['hospital_expire_flag'] == 0, 'y_label'] = 0

# Creating targets and balancing set)
case_df = visit_pts_df[visit_pts_df.y_label == 1]
control_df = visit_pts_df[visit_pts_df.y_label == 0]

### Write

In [6]:
case_df.to_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_case_df2.csv'}"),index_label=False)
control_df.to_csv(os.path.join('output', f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_control_df2.csv'}"),index_label=False)

### Read

In [7]:
# Read
case_df = pd.read_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_case_df2.csv'}"),index_col=0)
control_df = pd.read_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_control_df2.csv'}"),index_col=0)

### Train

In [ ]:
# Creating targets and balancing set)
visit_pts_df.loc[visit_pts_df['hospital_expire_flag'] == 1, 'y_label'] = 1
visit_pts_df.loc[visit_pts_df['dod'].isna(), 'y_label'] = 0
visit_pts_df.loc[visit_pts_df['hospital_expire_flag'] == 0, 'y_label'] = 0

# Creating targets and balancing set)
case_df = visit_pts_df[visit_pts_df.y_label == 1]
control_df = visit_pts_df[visit_pts_df.y_label == 0]

print(f"Positive cases: {len(case_df)}")
print(f"Negative cases: {len(control_df)}")

control_df = control_df.sample(len(case_df),random_state=42)

# Concatenate
ml_df = pd.concat([case_df, control_df], axis=0)
ml_df.reset_index(drop=True,inplace=True)

# Coerce variables to ints
ml_df['los_hours'] = ml_df['los_hours'].astype(int)
ml_df['age'] = ml_df['age'].astype(int)
ml_df['y_label'] = ml_df['y_label'].astype(int)
ml_df['comorbs'] = ml_df['comorbs'].astype(int)
ml_df['bmi'] = ml_df['bmi'].astype(float)
ml_df['bp_systolic'] = ml_df['bp_systolic'].astype(int)
ml_df['bp_diastolic'] = ml_df['bp_diastolic'].astype(int)
ml_df['presc_meds'] = ml_df['presc_meds'].astype(int)

# Create one-hot encodings for categorical features
categorical_columns = ['admission_type', 'admission_location',
       'insurance', 'ethnicity', 'marital_status', 'gender']  

encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(ml_df[categorical_columns])
feature_names = encoder.get_feature_names_out(categorical_columns)
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

# Drop original categorical columns and concatenate with encoded ones
ml_df = pd.concat([ml_df.drop(categorical_columns, axis=1), encoded_df], axis=1)

# Selecting only properly encoded columns
ml_df = ml_df[['los_hours',
       'age', 'y_label', 'bmi','comorbs','bp_systolic','bp_diastolic', 
       'presc_meds',
       'admission_type_AMBULATORY OBSERVATION', 'admission_type_DIRECT EMER.',
       'admission_type_DIRECT OBSERVATION', 'admission_type_ELECTIVE',
       'admission_type_EU OBSERVATION', 'admission_type_EW EMER.',
       'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGENT',
       'admission_location_AMBULATORY SURGERY TRANSFER',
       'admission_location_CLINIC REFERRAL',
       'admission_location_EMERGENCY ROOM',
       'admission_location_INFORMATION NOT AVAILABLE',
       'admission_location_PACU', 'admission_location_PHYSICIAN REFERRAL',
       'admission_location_PROCEDURE SITE',
       'admission_location_TRANSFER FROM HOSPITAL',
       'admission_location_TRANSFER FROM SKILLED NURSING FACILITY',
       'admission_location_WALK-IN/SELF REFERRAL', 'insurance_Medicaid',
       'insurance_Medicare', 'insurance_Other',
       'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN',
       'ethnicity_BLACK/AFRICAN AMERICAN',
       'ethnicity_OTHER', 
       'ethnicity_UNABLE TO OBTAIN',
       'ethnicity_UNKNOWN',
       'ethnicity_WHITE', 'marital_status_DIVORCED', 'marital_status_MARRIED',
       'marital_status_SINGLE', 'marital_status_WIDOWED', 'marital_status_nan',
       'gender_F', 'gender_M']]

X = ml_df.drop('y_label', axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = ml_df['y_label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# GS
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear','saga'],
    'tol': [1e-4, 1e-3, 1e-2], 
    'l1_ratio': [0.1, 0.3, 0.5, 0.7]
    }

# Create and fit grid search
grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000),
    param_grid,
    cv=5,
    scoring='f1_macro',
    refit=True,
    verbose=1
)
grid_search.fit(X_train, y_train)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.3f}") 

# Training quick model
#model = LogisticRegression(penalty='l2', max_iter=5000, C=0.1)
#model.fit(X_train, y_train)



### Eval

In [ ]:
# Predict
y_pred = grid_search.predict(X_test)
#y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name=f'LR')
display.plot()
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.show()

### Write resulting dataset

In [19]:
ml_df.to_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_ml_df2.csv'}"),index_label=False)

## Readmission task

### Creating partitions based on readmission gap

In [ ]:
# Task spec
task = "readmit_pred"
# These are expensive to run, so we write to 'output' folder
case_df, control_df, invalid_df, redundant_df = partition_by_readmit(visit_pts_df,gap=GAP,group_col=GROUP_COL,visit_col=VISIT_COL,admit_col=ADMIT_COL,disch_col=DISCH_COL)

### Write

In [22]:
case_df.to_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_case_df.csv'}"),index_label=False)
control_df.to_csv(os.path.join('output', f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_control_df.csv'}"),index_label=False)
invalid_df.to_csv(os.path.join('output', f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_invalid_df.csv'}"),index_label=False)
redundant_df.to_csv(os.path.join('output', f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_redundant_df.csv'}"),index_label=False)

### Read

In [9]:
# Read
case_df = pd.read_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_case_df.csv'}"),index_col=0)
control_df = pd.read_csv(os.path.join('output',f"{task + '_' + DISEASE_LABEL + '_GAP'+ str(GAP.days) + '_MIMIC_IV' + '_' + VERSION + '_control_df.csv'}"),index_col=0)

### Train

In [ ]:
# Creating targets and balancing set
case_df['y_label'] = np.array([1] * len(case_df))
control_df['y_label'] = np.array([0] * len(control_df))

print(f"Positive cases: {len(case_df)}")
print(f"Negative cases: {len(control_df)}")

control_df = control_df.sample(len(case_df),random_state=42)

# Concatenate
ml_df = pd.concat([case_df, control_df], axis=0)
ml_df.reset_index(drop=True,inplace=True)

# Coerce variables to ints
ml_df['los_hours'] = ml_df['los_hours'].astype(int)
ml_df['age'] = ml_df['age'].astype(int)
ml_df['y_label'] = ml_df['y_label'].astype(int)
ml_df['comorbs'] = ml_df['comorbs'].astype(int)
ml_df['bmi'] = ml_df['bmi'].astype(float)
ml_df['bp_systolic'] = ml_df['bp_systolic'].astype(int)
ml_df['bp_diastolic'] = ml_df['bp_diastolic'].astype(int)
ml_df['presc_meds'] = ml_df['presc_meds'].astype(int)

# Create one-hot encodings for categorical features
categorical_columns = ['admission_type', 'admission_location',
       'insurance', 'ethnicity', 'marital_status', 'gender']  

encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(ml_df[categorical_columns])
feature_names = encoder.get_feature_names_out(categorical_columns)
encoded_df = pd.DataFrame(encoded_features, columns=feature_names)

# Drop original categorical columns and concatenate with encoded ones
ml_df = pd.concat([ml_df.drop(categorical_columns, axis=1), encoded_df], axis=1)

# Z-score features
from sklearn.preprocessing import StandardScaler

# Selecting only properly encoded columns
ml_df = ml_df[['los_hours',
       'age', 'y_label', 'bmi','comorbs','bp_systolic','bp_diastolic', 
       'presc_meds',
       'admission_type_AMBULATORY OBSERVATION', 'admission_type_DIRECT EMER.',
       'admission_type_DIRECT OBSERVATION', 'admission_type_ELECTIVE',
       'admission_type_EU OBSERVATION', 'admission_type_EW EMER.',
       'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGENT',
       #'admission_location_AMBULATORY SURGERY TRANSFER',
       'admission_location_CLINIC REFERRAL',
       'admission_location_EMERGENCY ROOM',
       'admission_location_INFORMATION NOT AVAILABLE',
       'admission_location_PACU', 'admission_location_PHYSICIAN REFERRAL',
       'admission_location_PROCEDURE SITE',
       'admission_location_TRANSFER FROM HOSPITAL',
       'admission_location_TRANSFER FROM SKILLED NURSING FACILITY',
       'admission_location_WALK-IN/SELF REFERRAL', 'insurance_Medicaid',
       'insurance_Medicare', 'insurance_Other',
       'ethnicity_AMERICAN INDIAN/ALASKA NATIVE', 'ethnicity_ASIAN',
       'ethnicity_BLACK/AFRICAN AMERICAN',
       'ethnicity_OTHER', 
       #'ethnicity_UNABLE TO OBTAIN',
       'ethnicity_UNKNOWN',
       'ethnicity_WHITE', 'marital_status_DIVORCED', 'marital_status_MARRIED',
       'marital_status_SINGLE', 'marital_status_WIDOWED', 'marital_status_nan',
       'gender_F', 'gender_M']]

X = ml_df.drop('y_label', axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = ml_df['y_label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# GS
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10,],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear','saga'],
    'tol': [1e-4, 1e-3, 1e-2], 
    'l1_ratio': [0.1, 0.3, 0.5, 0.7]
    }

# Create and fit grid search
grid_search = GridSearchCV(
    LogisticRegression(max_iter=1000),
    param_grid,
    cv=5,
    scoring='f1_macro',
    refit=True,
    verbose=1
)
grid_search.fit(X_train, y_train)

# Print best parameters and score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.3f}") 

# Training quick model
#model = LogisticRegression(penalty='l2', max_iter=5000, C=2)
#model.fit(X_train, y_train)

### Eval

In [24]:
# Predict
y_pred = grid_search.predict(X_test)
#y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.55      0.52      0.53       261
           1       0.52      0.54      0.53       247

    accuracy                           0.53       508
   macro avg       0.53      0.53      0.53       508
weighted avg       0.53      0.53      0.53       508



In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

roc_auc = auc(fpr, tpr)
display = RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,
                                  estimator_name='LR')
display.plot()
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.show()

## Sanity check

In [ ]:
# Sanity checks
print(ml_df.info())

# Data integration

In [3]:
import pandas as pd
import os
from mimic_to_omop_mapper import MIMICToOMOPMapper

# Initialize the mapper
mapper = MIMICToOMOPMapper(output_dir=os.path.join("output", "OMOP_CDM"))

# Map your MIMIC data
mapper.map_mimic_data(os.path.join("output", "death_pred_I50_GAP14_MIMIC_IV_2.0_case_df.csv"))

Saved person with 1331 records to output/OMOP_CDM/person.csv
Saved measurement with 3993 records to output/OMOP_CDM/measurement.csv
Saved visit_occurrence with 1331 records to output/OMOP_CDM/visit_occurrence.csv


In [7]:
pd.read_csv(os.path.join("output", "death_pred_I50_GAP14_MIMIC_IV_2.0_case_df.csv")).head()

,subject_id,hadm_id,admittime,dischtime,los,los_hours,admission_type,admission_location,discharge_location,insurance,...,min_valid_year,dod,gender,age,comorbs,bmi,bp_systolic,bp_diastolic,presc_meds,y_label
29,10003400,23559586,2137-08-04 00:07:00,2137-09-02 17:05:00,29 days 16:58:00,713,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare,...,2140,2137-09-02,F,72,38,38.1,101.0,62.0,249,1.0
48,10004401,25777141,2144-06-05 19:45:00,2144-06-18 21:30:00,13 days 01:45:00,314,EW EMER.,EMERGENCY ROOM,DIED,Medicare,...,2150,2144-06-18,M,82,26,27.9,102.0,54.0,97,1.0
55,10005817,28661809,2135-01-03 21:54:00,2135-01-19 18:36:00,15 days 20:42:00,381,OBSERVATION ADMIT,TRANSFER FROM HOSPITAL,DIED,Medicare,...,2135,2135-01-19,M,66,25,30.5,153.0,66.0,241,1.0
65,10010058,26359957,2147-11-18 02:34:00,2147-11-19 04:00:00,1 days 01:26:00,25,EW EMER.,PHYSICIAN REFERRAL,DIED,Other,...,2145,2147-11-19,M,80,23,22.2,110.0,66.0,92,1.0
112,10015931,22130791,2177-03-24 21:47:00,2177-03-29 14:15:00,4 days 16:28:00,112,URGENT,TRANSFER FROM HOSPITAL,DIED,Medicare,...,2179,2177-03-29,M,87,23,20.0,122.0,60.0,87,1.0


In [4]:
pd.read_csv(os.path.join("output", "OMOP_CDM", "measurement.csv")).head()

,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,unit_concept_id,range_low,range_high,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value
0,5886870618654834761,10003400,3038553,2137-08-04,2137-08-04 00:07:00,44818701,4172703,38.1,NaN,9529,NaN,NaN,NaN,23559586,NaN,BMI,0,kg/m2,38.1
1,8015629750266426386,10003400,3004249,2137-08-04,2137-08-04 00:07:00,44818701,4172703,101.0,NaN,8876,NaN,NaN,NaN,23559586,NaN,Systolic BP,0,mmHg,101.0
2,7049997083418041962,10003400,3012888,2137-08-04,2137-08-04 00:07:00,44818701,4172703,62.0,NaN,8876,NaN,NaN,NaN,23559586,NaN,Diastolic BP,0,mmHg,62.0
3,7684553742813515083,10004401,3038553,2144-06-05,2144-06-05 19:45:00,44818701,4172703,27.9,NaN,9529,NaN,NaN,NaN,25777141,NaN,BMI,0,kg/m2,27.9
4,1274868572764232421,10004401,3004249,2144-06-05,2144-06-05 19:45:00,44818701,4172703,102.0,NaN,8876,NaN,NaN,NaN,25777141,NaN,Systolic BP,0,mmHg,102.0


In [5]:
pd.read_csv(os.path.join("output", "OMOP_CDM", "person.csv")).head()

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,10003400,8532,2062,NaN,NaN,2062-01-01,0,NaN,NaN,NaN,10003400,F,0,0,BLACK/AFRICAN AMERICAN,0
1,10004401,8507,2059,NaN,NaN,2059-01-01,0,NaN,NaN,NaN,10004401,M,0,0,WHITE,0
2,10005817,8507,2066,NaN,NaN,2066-01-01,0,NaN,NaN,NaN,10005817,M,0,0,WHITE,0
3,10010058,8507,2059,NaN,NaN,2059-01-01,0,NaN,NaN,NaN,10010058,M,0,0,ASIAN - SOUTH EAST ASIAN,0
4,10015931,8507,2089,NaN,NaN,2089-01-01,0,NaN,NaN,NaN,10015931,M,0,0,WHITE,0


In [6]:
pd.read_csv(os.path.join("output", "OMOP_CDM", "visit_occurrence.csv")).head()

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitted_from_concept_id,admitted_from_source_value,discharged_to_concept_id,discharged_to_source_value
0,23559586,10003400,9203,2137-08-04,2137-08-04 00:07:00,2137-09-02,2137-09-02 17:05:00,44818517,NaN,NaN,23559586,0,8863,TRANSFER FROM HOSPITAL,4216643,DIED
1,25777141,10004401,0,2144-06-05,2144-06-05 19:45:00,2144-06-18,2144-06-18 21:30:00,44818517,NaN,NaN,25777141,0,8870,EMERGENCY ROOM,4216643,DIED
2,28661809,10005817,0,2135-01-03,2135-01-03 21:54:00,2135-01-19,2135-01-19 18:36:00,44818517,NaN,NaN,28661809,0,8863,TRANSFER FROM HOSPITAL,4216643,DIED
3,26359957,10010058,0,2147-11-18,2147-11-18 02:34:00,2147-11-19,2147-11-19 04:00:00,44818517,NaN,NaN,26359957,0,8870,PHYSICIAN REFERRAL,4216643,DIED
4,22130791,10015931,9203,2177-03-24,2177-03-24 21:47:00,2177-03-29,2177-03-29 14:15:00,44818517,NaN,NaN,22130791,0,8863,TRANSFER FROM HOSPITAL,4216643,DIED
